In [18]:
import dotenv
import os

import requests
import json

from datetime import datetime

In [23]:
dotenv_file = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_file)

url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'

now = datetime.now()
today = datetime.today().strftime('%Y%m%d')
nx = 60
ny = 127

params ={'serviceKey' : os.environ["decode_key"], 'pageNo' : '1', 'numOfRows' : '1000', 'dataType' : 'JSON', 'base_date' : today, 'base_time' : '0600', 'nx' : nx, 'ny' : ny }

res = requests.get(url, params=params)
result = json.loads(res.content)
items = result['response']['body']['items']['item']
# print(items)

for i in items :
  print(i['category'])

PTY
REH
RN1
T1H
UUU
VEC
VVV
WSD
